In [12]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, RobustScaler

from sklearn import set_config
set_config(display="diagram")

import time

In [13]:
house_data = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/housing-classification-iter6.csv')
house_data

,LotArea,LotFrontage,TotalBsmtSF,BedroomAbvGr,Fireplaces,PoolArea,GarageCars,WoodDeckSF,ScreenPorch,Expensive,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,8450,65.0,856,3,0,0,2,0,0,0,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
1,9600,80.0,1262,3,1,0,2,298,0,0,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
2,11250,68.0,920,3,1,0,2,0,0,0,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
3,9550,60.0,756,3,1,0,3,0,0,0,...,Detchd,Unf,TA,TA,Y,NaN,NaN,NaN,WD,Abnorml
4,14260,84.0,1145,4,1,0,3,192,0,0,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,7917,62.0,953,3,1,0,2,0,0,0,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
1456,13175,85.0,1542,3,2,0,2,349,0,0,...,Attchd,Unf,TA,TA,Y,NaN,MnPrv,NaN,WD,Normal
1457,9042,66.0,1152,4,2,0,1,0,0,1,...,Attchd,RFn,TA,TA,Y,NaN,GdPrv,Shed,WD,Normal
1458,9717,68.0,1078,2,0,0,1,366,0,0,...,Attchd,Unf,TA,TA,Y,NaN,NaN,NaN,WD,Normal


In [14]:
house_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   LotArea        1460 non-null   int64  
 1   LotFrontage    1201 non-null   float64
 2   TotalBsmtSF    1460 non-null   int64  
 3   BedroomAbvGr   1460 non-null   int64  
 4   Fireplaces     1460 non-null   int64  
 5   PoolArea       1460 non-null   int64  
 6   GarageCars     1460 non-null   int64  
 7   WoodDeckSF     1460 non-null   int64  
 8   ScreenPorch    1460 non-null   int64  
 9   Expensive      1460 non-null   int64  
 10  MSZoning       1460 non-null   object 
 11  Condition1     1460 non-null   object 
 12  Heating        1460 non-null   object 
 13  Street         1460 non-null   object 
 14  CentralAir     1460 non-null   object 
 15  Foundation     1460 non-null   object 
 16  ExterQual      1460 non-null   object 
 17  ExterCond      1460 non-null   object 
 18  BsmtQual

In [15]:
y = house_data.pop('Expensive')
y

0       0
1       0
2       0
3       0
4       0
       ..
1455    0
1456    0
1457    1
1458    0
1459    0
Name: Expensive, Length: 1460, dtype: int64

In [16]:
y.value_counts()

0    1243
1     217
Name: Expensive, dtype: int64

##### **Prepare the data categories into numerical and categorical data**

In [17]:
X_num = house_data.select_dtypes(include='number')
X_num

,LotArea,LotFrontage,TotalBsmtSF,BedroomAbvGr,Fireplaces,PoolArea,GarageCars,WoodDeckSF,ScreenPorch,MSSubClass,...,TotRmsAbvGrd,GarageYrBlt,GarageArea,OpenPorchSF,EnclosedPorch,3SsnPorch,MiscVal,MoSold,YrSold,Id
0,8450,65.0,856,3,0,0,2,0,0,60,...,8,2003.0,548,61,0,0,0,2,2008,1
1,9600,80.0,1262,3,1,0,2,298,0,20,...,6,1976.0,460,0,0,0,0,5,2007,2
2,11250,68.0,920,3,1,0,2,0,0,60,...,6,2001.0,608,42,0,0,0,9,2008,3
3,9550,60.0,756,3,1,0,3,0,0,70,...,7,1998.0,642,35,272,0,0,2,2006,4
4,14260,84.0,1145,4,1,0,3,192,0,60,...,9,2000.0,836,84,0,0,0,12,2008,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,7917,62.0,953,3,1,0,2,0,0,60,...,7,1999.0,460,40,0,0,0,8,2007,1456
1456,13175,85.0,1542,3,2,0,2,349,0,20,...,7,1978.0,500,0,0,0,0,2,2010,1457
1457,9042,66.0,1152,4,2,0,1,0,0,70,...,9,1941.0,252,60,0,0,2500,5,2010,1458
1458,9717,68.0,1078,2,0,0,1,366,0,20,...,5,1950.0,240,0,112,0,0,4,2010,1459


In [18]:
X_num.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 37 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   LotArea        1460 non-null   int64  
 1   LotFrontage    1201 non-null   float64
 2   TotalBsmtSF    1460 non-null   int64  
 3   BedroomAbvGr   1460 non-null   int64  
 4   Fireplaces     1460 non-null   int64  
 5   PoolArea       1460 non-null   int64  
 6   GarageCars     1460 non-null   int64  
 7   WoodDeckSF     1460 non-null   int64  
 8   ScreenPorch    1460 non-null   int64  
 9   MSSubClass     1460 non-null   int64  
 10  OverallQual    1460 non-null   int64  
 11  OverallCond    1460 non-null   int64  
 12  YearBuilt      1460 non-null   int64  
 13  YearRemodAdd   1460 non-null   int64  
 14  MasVnrArea     1452 non-null   float64
 15  BsmtFinSF1     1460 non-null   int64  
 16  BsmtFinSF2     1460 non-null   int64  
 17  BsmtUnfSF      1460 non-null   int64  
 18  1stFlrSF

In [19]:
X_cat = house_data.select_dtypes(include='object')
X_cat

,MSZoning,Condition1,Heating,Street,CentralAir,Foundation,ExterQual,ExterCond,BsmtQual,BsmtCond,...,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,RL,Norm,GasA,Pave,Y,PConc,Gd,TA,Gd,TA,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
1,RL,Feedr,GasA,Pave,Y,CBlock,TA,TA,Gd,TA,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
2,RL,Norm,GasA,Pave,Y,PConc,Gd,TA,Gd,TA,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
3,RL,Norm,GasA,Pave,Y,BrkTil,TA,TA,TA,Gd,...,Detchd,Unf,TA,TA,Y,NaN,NaN,NaN,WD,Abnorml
4,RL,Norm,GasA,Pave,Y,PConc,Gd,TA,Gd,TA,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,RL,Norm,GasA,Pave,Y,PConc,TA,TA,Gd,TA,...,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
1456,RL,Norm,GasA,Pave,Y,CBlock,TA,TA,Gd,TA,...,Attchd,Unf,TA,TA,Y,NaN,MnPrv,NaN,WD,Normal
1457,RL,Norm,GasA,Pave,Y,Stone,Ex,Gd,TA,Gd,...,Attchd,RFn,TA,TA,Y,NaN,GdPrv,Shed,WD,Normal
1458,RL,Norm,GasA,Pave,Y,CBlock,TA,TA,TA,TA,...,Attchd,Unf,TA,TA,Y,NaN,NaN,NaN,WD,Normal


In [20]:
X_cat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 43 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   MSZoning       1460 non-null   object
 1   Condition1     1460 non-null   object
 2   Heating        1460 non-null   object
 3   Street         1460 non-null   object
 4   CentralAir     1460 non-null   object
 5   Foundation     1460 non-null   object
 6   ExterQual      1460 non-null   object
 7   ExterCond      1460 non-null   object
 8   BsmtQual       1423 non-null   object
 9   BsmtCond       1423 non-null   object
 10  BsmtExposure   1422 non-null   object
 11  BsmtFinType1   1423 non-null   object
 12  KitchenQual    1460 non-null   object
 13  FireplaceQu    770 non-null    object
 14  Alley          91 non-null     object
 15  LotShape       1460 non-null   object
 16  LandContour    1460 non-null   object
 17  Utilities      1460 non-null   object
 18  LotConfig      1460 non-null

In [21]:
# Investigate each feature in the numerical data.
X_num['MasVnrArea'].value_counts()
# drop PoolArea, ScreenPorch, LowQualFinSF, BsmtHalfBath, EnclosedPorch, 3SsnPorch, MiscVal, Id; they have no substantial variability.
# convert all date entries and MSSubClass to one-hot categorical features.

0.0      861
180.0      8
72.0       8
108.0      8
120.0      7
        ... 
562.0      1
89.0       1
921.0      1
762.0      1
119.0      1
Name: MasVnrArea, Length: 327, dtype: int64

In [22]:
X_cat['MoSold'] = X_num.pop('MoSold')
X_cat['MoSold'] = X_cat['MoSold'].astype(str)

X_cat['YrSold'] = X_num.pop('YrSold')
X_cat['YrSold'] = X_cat['YrSold'].astype(str)

X_cat['YearBuilt'] = X_num.pop('YearBuilt')
X_cat['YearBuilt'] = X_cat['YearBuilt'].astype(str)

X_cat['YearRemodAdd'] = X_num.pop('YearRemodAdd')
X_cat['YearRemodAdd'] = X_cat['YearRemodAdd'].astype(str)

X_cat['GarageYrBlt'] = X_num.pop('GarageYrBlt')
X_cat['GarageYrBlt'] = X_cat['GarageYrBlt'].astype(str)

X_cat['MSSubClass'] = X_num.pop('MSSubClass')
X_cat['MSSubClass'] = X_cat['MSSubClass'].astype(str)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documen

In [23]:
X_num = X_num.drop(['PoolArea', 'ScreenPorch', 'LowQualFinSF',\
                    'BsmtHalfBath', 'EnclosedPorch', '3SsnPorch', 'MiscVal', 'Id'], axis=1)

X_num

,LotArea,LotFrontage,TotalBsmtSF,BedroomAbvGr,Fireplaces,GarageCars,WoodDeckSF,OverallQual,OverallCond,MasVnrArea,...,1stFlrSF,2ndFlrSF,GrLivArea,BsmtFullBath,FullBath,HalfBath,KitchenAbvGr,TotRmsAbvGrd,GarageArea,OpenPorchSF
0,8450,65.0,856,3,0,2,0,7,5,196.0,...,856,854,1710,1,2,1,1,8,548,61
1,9600,80.0,1262,3,1,2,298,6,8,0.0,...,1262,0,1262,0,2,0,1,6,460,0
2,11250,68.0,920,3,1,2,0,7,5,162.0,...,920,866,1786,1,2,1,1,6,608,42
3,9550,60.0,756,3,1,3,0,7,5,0.0,...,961,756,1717,1,1,0,1,7,642,35
4,14260,84.0,1145,4,1,3,192,8,5,350.0,...,1145,1053,2198,1,2,1,1,9,836,84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,7917,62.0,953,3,1,2,0,6,5,0.0,...,953,694,1647,0,2,1,1,7,460,40
1456,13175,85.0,1542,3,2,2,349,6,6,119.0,...,2073,0,2073,1,2,0,1,7,500,0
1457,9042,66.0,1152,4,2,1,0,7,9,0.0,...,1188,1152,2340,0,2,0,1,9,252,60
1458,9717,68.0,1078,2,0,1,366,5,6,0.0,...,1078,0,1078,1,1,0,1,5,240,0


In [24]:
X_num.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 23 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   LotArea       1460 non-null   int64  
 1   LotFrontage   1201 non-null   float64
 2   TotalBsmtSF   1460 non-null   int64  
 3   BedroomAbvGr  1460 non-null   int64  
 4   Fireplaces    1460 non-null   int64  
 5   GarageCars    1460 non-null   int64  
 6   WoodDeckSF    1460 non-null   int64  
 7   OverallQual   1460 non-null   int64  
 8   OverallCond   1460 non-null   int64  
 9   MasVnrArea    1452 non-null   float64
 10  BsmtFinSF1    1460 non-null   int64  
 11  BsmtFinSF2    1460 non-null   int64  
 12  BsmtUnfSF     1460 non-null   int64  
 13  1stFlrSF      1460 non-null   int64  
 14  2ndFlrSF      1460 non-null   int64  
 15  GrLivArea     1460 non-null   int64  
 16  BsmtFullBath  1460 non-null   int64  
 17  FullBath      1460 non-null   int64  
 18  HalfBath      1460 non-null 

In [25]:
X_cat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 49 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   MSZoning       1460 non-null   object
 1   Condition1     1460 non-null   object
 2   Heating        1460 non-null   object
 3   Street         1460 non-null   object
 4   CentralAir     1460 non-null   object
 5   Foundation     1460 non-null   object
 6   ExterQual      1460 non-null   object
 7   ExterCond      1460 non-null   object
 8   BsmtQual       1423 non-null   object
 9   BsmtCond       1423 non-null   object
 10  BsmtExposure   1422 non-null   object
 11  BsmtFinType1   1423 non-null   object
 12  KitchenQual    1460 non-null   object
 13  FireplaceQu    770 non-null    object
 14  Alley          91 non-null     object
 15  LotShape       1460 non-null   object
 16  LandContour    1460 non-null   object
 17  Utilities      1460 non-null   object
 18  LotConfig      1460 non-null

In [26]:
X_cat_ord = X_cat[['GarageCond', 'GarageFinish', 'GarageQual', 'Electrical', 'HeatingQC', 'BsmtFinType2',\
                   'BsmtFinType1', 'BsmtExposure', 'KitchenQual', 'BsmtQual', 'BsmtCond', 'ExterCond', 'ExterQual', 'Functional']]

X_cat_ord

,GarageCond,GarageFinish,GarageQual,Electrical,HeatingQC,BsmtFinType2,BsmtFinType1,BsmtExposure,KitchenQual,BsmtQual,BsmtCond,ExterCond,ExterQual,Functional
0,TA,RFn,TA,SBrkr,Ex,Unf,GLQ,No,Gd,Gd,TA,TA,Gd,Typ
1,TA,RFn,TA,SBrkr,Ex,Unf,ALQ,Gd,TA,Gd,TA,TA,TA,Typ
2,TA,RFn,TA,SBrkr,Ex,Unf,GLQ,Mn,Gd,Gd,TA,TA,Gd,Typ
3,TA,Unf,TA,SBrkr,Gd,Unf,ALQ,No,Gd,TA,Gd,TA,TA,Typ
4,TA,RFn,TA,SBrkr,Ex,Unf,GLQ,Av,Gd,Gd,TA,TA,Gd,Typ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,TA,RFn,TA,SBrkr,Ex,Unf,Unf,No,TA,Gd,TA,TA,TA,Typ
1456,TA,Unf,TA,SBrkr,TA,Rec,ALQ,No,TA,Gd,TA,TA,TA,Min1
1457,TA,RFn,TA,SBrkr,Ex,Unf,GLQ,No,Gd,TA,Gd,Gd,Ex,Typ
1458,TA,Unf,TA,FuseA,Gd,Rec,GLQ,Mn,Gd,TA,TA,TA,TA,Typ


In [27]:
X_cat_onehot = X_cat.drop(['Fence', 'GarageCond', 'GarageFinish', 'GarageQual', 'Electrical', 'HeatingQC', 'BsmtFinType2', 'FireplaceQu', 'Alley', \
                           'BsmtFinType1', 'BsmtExposure', 'KitchenQual', 'BsmtQual', 'BsmtCond', 'ExterCond', 'ExterQual', 'PoolQC', 'MiscFeature', 'Functional'], axis=1)

X_cat_onehot

,MSZoning,Condition1,Heating,Street,CentralAir,Foundation,LotShape,LandContour,Utilities,LotConfig,...,GarageType,PavedDrive,SaleType,SaleCondition,MoSold,YrSold,YearBuilt,YearRemodAdd,GarageYrBlt,MSSubClass
0,RL,Norm,GasA,Pave,Y,PConc,Reg,Lvl,AllPub,Inside,...,Attchd,Y,WD,Normal,2,2008,2003,2003,2003.0,60
1,RL,Feedr,GasA,Pave,Y,CBlock,Reg,Lvl,AllPub,FR2,...,Attchd,Y,WD,Normal,5,2007,1976,1976,1976.0,20
2,RL,Norm,GasA,Pave,Y,PConc,IR1,Lvl,AllPub,Inside,...,Attchd,Y,WD,Normal,9,2008,2001,2002,2001.0,60
3,RL,Norm,GasA,Pave,Y,BrkTil,IR1,Lvl,AllPub,Corner,...,Detchd,Y,WD,Abnorml,2,2006,1915,1970,1998.0,70
4,RL,Norm,GasA,Pave,Y,PConc,IR1,Lvl,AllPub,FR2,...,Attchd,Y,WD,Normal,12,2008,2000,2000,2000.0,60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,RL,Norm,GasA,Pave,Y,PConc,Reg,Lvl,AllPub,Inside,...,Attchd,Y,WD,Normal,8,2007,1999,2000,1999.0,60
1456,RL,Norm,GasA,Pave,Y,CBlock,Reg,Lvl,AllPub,Inside,...,Attchd,Y,WD,Normal,2,2010,1978,1988,1978.0,20
1457,RL,Norm,GasA,Pave,Y,Stone,Reg,Lvl,AllPub,Inside,...,Attchd,Y,WD,Normal,5,2010,1941,2006,1941.0,70
1458,RL,Norm,GasA,Pave,Y,CBlock,Reg,Lvl,AllPub,Inside,...,Attchd,Y,WD,Normal,4,2010,1950,1996,1950.0,20


In [28]:
X_cat_ord.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   GarageCond    1379 non-null   object
 1   GarageFinish  1379 non-null   object
 2   GarageQual    1379 non-null   object
 3   Electrical    1459 non-null   object
 4   HeatingQC     1460 non-null   object
 5   BsmtFinType2  1422 non-null   object
 6   BsmtFinType1  1423 non-null   object
 7   BsmtExposure  1422 non-null   object
 8   KitchenQual   1460 non-null   object
 9   BsmtQual      1423 non-null   object
 10  BsmtCond      1423 non-null   object
 11  ExterCond     1460 non-null   object
 12  ExterQual     1460 non-null   object
 13  Functional    1460 non-null   object
dtypes: object(14)
memory usage: 159.8+ KB


In [29]:
X_cat_onehot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 30 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   MSZoning       1460 non-null   object
 1   Condition1     1460 non-null   object
 2   Heating        1460 non-null   object
 3   Street         1460 non-null   object
 4   CentralAir     1460 non-null   object
 5   Foundation     1460 non-null   object
 6   LotShape       1460 non-null   object
 7   LandContour    1460 non-null   object
 8   Utilities      1460 non-null   object
 9   LotConfig      1460 non-null   object
 10  LandSlope      1460 non-null   object
 11  Neighborhood   1460 non-null   object
 12  Condition2     1460 non-null   object
 13  BldgType       1460 non-null   object
 14  HouseStyle     1460 non-null   object
 15  RoofStyle      1460 non-null   object
 16  RoofMatl       1460 non-null   object
 17  Exterior1st    1460 non-null   object
 18  Exterior2nd    1460 non-null

##### **Adjust certain numerical features**

In [30]:
X_num[X_num.MasVnrArea.isna()]

,LotArea,LotFrontage,TotalBsmtSF,BedroomAbvGr,Fireplaces,GarageCars,WoodDeckSF,OverallQual,OverallCond,MasVnrArea,...,1stFlrSF,2ndFlrSF,GrLivArea,BsmtFullBath,FullBath,HalfBath,KitchenAbvGr,TotRmsAbvGrd,GarageArea,OpenPorchSF
234,7851,NaN,860,4,2,2,288,6,5,NaN,...,860,1100,1960,1,2,1,1,8,440,48
529,32668,NaN,2035,4,2,2,0,6,3,NaN,...,2515,0,2515,1,3,0,2,9,484,0
650,8125,65.0,813,3,0,2,0,7,6,NaN,...,822,843,1665,0,2,1,1,7,562,0
936,10083,67.0,1176,2,0,2,0,7,5,NaN,...,1200,0,1200,1,2,0,1,5,555,41
973,11639,95.0,1428,3,0,2,0,7,5,NaN,...,1428,0,1428,0,2,0,1,6,480,120
977,4274,35.0,1241,1,0,2,0,7,5,NaN,...,1241,0,1241,1,1,1,1,4,569,116
1243,13891,107.0,2076,2,1,3,216,10,5,NaN,...,2076,0,2076,1,2,1,1,7,850,229
1278,9473,75.0,1128,3,1,2,0,8,5,NaN,...,1128,903,2031,1,2,1,1,7,577,211


In [31]:
filler = int(np.round(np.median([x for x in X_num.MasVnrArea if (x != 0) & (x == x)])))

for x in list(X_num[X_num.MasVnrArea.isna()].index):
    X_num.loc[x, 'MasVnrArea'] = filler

In [32]:
X_num[X_num.MasVnrArea.isna()]

,LotArea,LotFrontage,TotalBsmtSF,BedroomAbvGr,Fireplaces,GarageCars,WoodDeckSF,OverallQual,OverallCond,MasVnrArea,...,1stFlrSF,2ndFlrSF,GrLivArea,BsmtFullBath,FullBath,HalfBath,KitchenAbvGr,TotRmsAbvGrd,GarageArea,OpenPorchSF


In [33]:
X_num.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 23 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   LotArea       1460 non-null   int64  
 1   LotFrontage   1201 non-null   float64
 2   TotalBsmtSF   1460 non-null   int64  
 3   BedroomAbvGr  1460 non-null   int64  
 4   Fireplaces    1460 non-null   int64  
 5   GarageCars    1460 non-null   int64  
 6   WoodDeckSF    1460 non-null   int64  
 7   OverallQual   1460 non-null   int64  
 8   OverallCond   1460 non-null   int64  
 9   MasVnrArea    1460 non-null   float64
 10  BsmtFinSF1    1460 non-null   int64  
 11  BsmtFinSF2    1460 non-null   int64  
 12  BsmtUnfSF     1460 non-null   int64  
 13  1stFlrSF      1460 non-null   int64  
 14  2ndFlrSF      1460 non-null   int64  
 15  GrLivArea     1460 non-null   int64  
 16  BsmtFullBath  1460 non-null   int64  
 17  FullBath      1460 non-null   int64  
 18  HalfBath      1460 non-null 

##### **Check the numerical data**

In [34]:
numeric_pipeline = make_pipeline(SimpleImputer(), RobustScaler())
numeric_pipeline

Pipeline(steps=[('simpleimputer', SimpleImputer()),
                ('robustscaler', RobustScaler())])

In [35]:
full_pipeline = make_pipeline(numeric_pipeline, DecisionTreeClassifier())
full_pipeline

Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('simpleimputer', SimpleImputer()),
                                 ('robustscaler', RobustScaler())])),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [36]:
param_grid = {'decisiontreeclassifier__criterion': ['gini', 'entropy'],
              'decisiontreeclassifier__max_depth': range(2, 8, 2),
              'decisiontreeclassifier__min_samples_leaf': range(3, 5, 2),
              'decisiontreeclassifier__min_samples_split': range(5, 15, 5)
             }
param_grid

{'decisiontreeclassifier__criterion': ['gini', 'entropy'],
 'decisiontreeclassifier__max_depth': range(2, 8, 2),
 'decisiontreeclassifier__min_samples_leaf': range(3, 5, 2),
 'decisiontreeclassifier__min_samples_split': range(5, 15, 5)}

In [37]:
search = GridSearchCV(full_pipeline, param_grid, cv=5, verbose=1)
search

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('pipeline',
                                        Pipeline(steps=[('simpleimputer',
                                                         SimpleImputer()),
                                                        ('robustscaler',
                                                         RobustScaler())])),
                                       ('decisiontreeclassifier',
                                        DecisionTreeClassifier())]),
             param_grid={'decisiontreeclassifier__criterion': ['gini',
                                                               'entropy'],
                         'decisiontreeclassifier__max_depth': range(2, 8, 2),
                         'decisiontreeclassifier__min_samples_leaf': range(3, 5, 2),
                         'decisiontreeclassifier__min_samples_split': range(5, 15, 5)},
             verbose=1)

In [38]:
search.fit(X_num, y)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('pipeline',
                                        Pipeline(steps=[('simpleimputer',
                                                         SimpleImputer()),
                                                        ('robustscaler',
                                                         RobustScaler())])),
                                       ('decisiontreeclassifier',
                                        DecisionTreeClassifier())]),
             param_grid={'decisiontreeclassifier__criterion': ['gini',
                                                               'entropy'],
                         'decisiontreeclassifier__max_depth': range(2, 8, 2),
                         'decisiontreeclassifier__min_samples_leaf': range(3, 5, 2),
                         'decisiontreeclassifier__min_samples_split': range(5, 15, 5)},
             verbose=1)

##### **Check Ordinal Categorical Data**

In [39]:
X_cat_ord.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   GarageCond    1379 non-null   object
 1   GarageFinish  1379 non-null   object
 2   GarageQual    1379 non-null   object
 3   Electrical    1459 non-null   object
 4   HeatingQC     1460 non-null   object
 5   BsmtFinType2  1422 non-null   object
 6   BsmtFinType1  1423 non-null   object
 7   BsmtExposure  1422 non-null   object
 8   KitchenQual   1460 non-null   object
 9   BsmtQual      1423 non-null   object
 10  BsmtCond      1423 non-null   object
 11  ExterCond     1460 non-null   object
 12  ExterQual     1460 non-null   object
 13  Functional    1460 non-null   object
dtypes: object(14)
memory usage: 159.8+ KB


In [40]:
X_cat_ord.Electrical.value_counts()

SBrkr    1334
FuseA      94
FuseF      27
FuseP       3
Mix         1
Name: Electrical, dtype: int64

In [41]:
X_cat_ord[X_cat_ord.Electrical.isna()]

,GarageCond,GarageFinish,GarageQual,Electrical,HeatingQC,BsmtFinType2,BsmtFinType1,BsmtExposure,KitchenQual,BsmtQual,BsmtCond,ExterCond,ExterQual,Functional
1379,TA,Fin,TA,NaN,Gd,Unf,Unf,No,Gd,Gd,TA,TA,TA,Typ


In [42]:
X_cat_ord.loc[1379, 'Electrical'] = 'Mix'

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [43]:
X_cat_ord[X_cat_ord.Electrical.isna()]

,GarageCond,GarageFinish,GarageQual,Electrical,HeatingQC,BsmtFinType2,BsmtFinType1,BsmtExposure,KitchenQual,BsmtQual,BsmtCond,ExterCond,ExterQual,Functional


In [44]:
categ_a = ['N_A', 'NA', 'Po', 'Fa', 'TA', 'Gd', 'Ex']
categ_b = ['N_A', 'NA', 'Unf', 'RFn', 'Fin']
categ_c = ['N_A', 'NA', 'Po', 'Fa', 'TA', 'Gd', 'Ex']
categ_d = ['Mix', 'FuseP', 'FuseF', 'FuseA', 'SBrkr']
categ_e = ['Po', 'Fa', 'TA', 'Gd', 'Ex']
categ_f = ['N_A', 'NA', 'Unf', 'LwQ', 'Rec', 'BLQ', 'ALQ', 'GLQ']
categ_g = ['N_A', 'NA', 'Unf', 'LwQ', 'Rec', 'BLQ', 'ALQ', 'GLQ']
categ_h = ['N_A', 'NA', 'No', 'Mn', 'Av', 'Gd']
categ_i = ['Po', 'Fa', 'TA', 'Gd', 'Ex']
categ_j = ['N_A', 'NA', 'Po', 'Fa', 'TA', 'Gd', 'Ex']
categ_k = ['N_A', 'NA', 'Po', 'Fa', 'TA', 'Gd', 'Ex']
categ_l = ['Po', 'Fa', 'TA', 'Gd', 'Ex']
categ_m = ['Po', 'Fa', 'TA', 'Gd', 'Ex']
categ_n = ['Sal', 'Sev', 'Maj2', 'Maj1', 'Mod', 'Min2', 'Min1', 'Typ']


cat_ord_categ = [categ_a, categ_b, categ_c, categ_d, categ_e, categ_f, categ_g, \
                 categ_h, categ_i, categ_j, categ_k, categ_l, categ_m, categ_n]

In [45]:
cat_ordinal_pipeline = make_pipeline(SimpleImputer(strategy="constant", fill_value="N_A"),\
                                     OrdinalEncoder(categories=cat_ord_categ))
cat_ordinal_pipeline

Pipeline(steps=[('simpleimputer',
                 SimpleImputer(fill_value='N_A', strategy='constant')),
                ('ordinalencoder',
                 OrdinalEncoder(categories=[['N_A', 'NA', 'Po', 'Fa', 'TA',
                                             'Gd', 'Ex'],
                                            ['N_A', 'NA', 'Unf', 'RFn', 'Fin'],
                                            ['N_A', 'NA', 'Po', 'Fa', 'TA',
                                             'Gd', 'Ex'],
                                            ['Mix', 'FuseP', 'FuseF', 'FuseA',
                                             'SBrkr'],
                                            ['Po', 'Fa', 'TA', 'Gd', 'Ex'],
                                            ['N_A', 'NA', 'Unf', 'LwQ', 'Rec',
                                             'BLQ', 'ALQ', 'GLQ'],
                                            ['N_A', 'NA', 'Unf', 'LwQ', 'Rec',
                                             'BLQ', 'ALQ', 'GLQ'],
                                            ['N_A', 'NA', 'No', 'Mn', 'Av',
                                             'Gd'],
                                            ['Po', 'Fa', 'TA', 'Gd', 'Ex'],
                                            ['N_A', 'NA', 'Po', 'Fa', 'TA',
                                             'Gd', 'Ex'],
                                            ['N_A', 'NA', 'Po', 'Fa', 'TA',
                                             'Gd', 'Ex'],
                                            ['Po', 'Fa', 'TA', 'Gd', 'Ex'],
                                            ['Po', 'Fa', 'TA', 'Gd', 'Ex'],
                                            ['Sal', 'Sev', 'Maj2', 'Maj1',
                                             'Mod', 'Min2', 'Min1', 'Typ']]))])

In [46]:
full_pipeline = make_pipeline(cat_ordinal_pipeline, DecisionTreeClassifier())
full_pipeline

Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('simpleimputer',
                                  SimpleImputer(fill_value='N_A',
                                                strategy='constant')),
                                 ('ordinalencoder',
                                  OrdinalEncoder(categories=[['N_A', 'NA', 'Po',
                                                              'Fa', 'TA', 'Gd',
                                                              'Ex'],
                                                             ['N_A', 'NA',
                                                              'Unf', 'RFn',
                                                              'Fin'],
                                                             ['N_A', 'NA', 'Po',
                                                              'Fa', 'TA', 'Gd',
                                                              'Ex'],
                                                             ['Mix', 'FuseP',
                                                              'FuseF', 'FuseA',
                                                              'SBrkr'],
                                                             ['Po', 'Fa', 'TA',
                                                              'Gd', 'Ex'],
                                                             ['N_A', 'NA',
                                                              'Unf'...
                                                             ['N_A', 'NA',
                                                              'Unf', 'LwQ',
                                                              'Rec', 'BLQ',
                                                              'ALQ', 'GLQ'],
                                                             ['N_A', 'NA', 'No',
                                                              'Mn', 'Av',
                                                              'Gd'],
                                                             ['Po', 'Fa', 'TA',
                                                              'Gd', 'Ex'],
                                                             ['N_A', 'NA', 'Po',
                                                              'Fa', 'TA', 'Gd',
                                                              'Ex'],
                                                             ['N_A', 'NA', 'Po',
                                                              'Fa', 'TA', 'Gd',
                                                              'Ex'],
                                                             ['Po', 'Fa', 'TA',
                                                              'Gd', 'Ex'],
                                                             ['Po', 'Fa', 'TA',
                                                              'Gd', 'Ex'],
                                                             ['Sal', 'Sev',
                                                              'Maj2', 'Maj1',
                                                              'Mod', 'Min2',
                                                              'Min1',
                                                              'Typ']]))])),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [47]:
param_grid = {'decisiontreeclassifier__criterion': ['gini', 'entropy'],
              'decisiontreeclassifier__max_depth': range(2, 8, 2),
              'decisiontreeclassifier__min_samples_leaf': range(3, 5, 2),
              'decisiontreeclassifier__min_samples_split': range(5, 15, 5)
             }
param_grid

{'decisiontreeclassifier__criterion': ['gini', 'entropy'],
 'decisiontreeclassifier__max_depth': range(2, 8, 2),
 'decisiontreeclassifier__min_samples_leaf': range(3, 5, 2),
 'decisiontreeclassifier__min_samples_split': range(5, 15, 5)}

In [48]:
search = GridSearchCV(full_pipeline, param_grid, cv=5, verbose=1)
search

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('pipeline',
                                        Pipeline(steps=[('simpleimputer',
                                                         SimpleImputer(fill_value='N_A',
                                                                       strategy='constant')),
                                                        ('ordinalencoder',
                                                         OrdinalEncoder(categories=[['N_A',
                                                                                     'NA',
                                                                                     'Po',
                                                                                     'Fa',
                                                                                     'TA',
                                                                                     'Gd',
                                                                                     'Ex'],
                                                                                    ['N_A',
                                                                                     'NA',
                                                                                     'Unf',
                                                                                     'RFn',
                                                                                     'Fin'],
                                                                                    ['N_A',
                                                                                     'NA',
                                                                                     'Po',
                                                                                     'Fa',
                                                                                     'TA',
                                                                                     'Gd',
                                                                                     'Ex'],
                                                                                    ['Mix',
                                                                                     'FuseP',
                                                                                     'FuseF',
                                                                                     'FuseA',
                                                                                     'SBrkr'],
                                                                                    ['Po',
                                                                                     'Fa',
                                                                                     'TA',...
                                                                                    ['Sal',
                                                                                     'Sev',
                                                                                     'Maj2',
                                                                                     'Maj1',
                                                                                     'Mod',
                                                                                     'Min2',
                                                                                     'Min1',
                                                                                     'Typ']]))])),
                                       ('decisiontreeclassifier',
                                        DecisionTreeClassifier())]),
             param_grid={'decisiontreeclassifier__criterion': ['gini',
                                                               'entropy'],
                         'decisiontreeclassifier__max_depth': range(2, 8, 2),
        

In [49]:
search.fit(X_cat_ord, y)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('pipeline',
                                        Pipeline(steps=[('simpleimputer',
                                                         SimpleImputer(fill_value='N_A',
                                                                       strategy='constant')),
                                                        ('ordinalencoder',
                                                         OrdinalEncoder(categories=[['N_A',
                                                                                     'NA',
                                                                                     'Po',
                                                                                     'Fa',
                                                                                     'TA',
                                                                                     'Gd',
                                                                                     'Ex'],
                                                                                    ['N_A',
                                                                                     'NA',
                                                                                     'Unf',
                                                                                     'RFn',
                                                                                     'Fin'],
                                                                                    ['N_A',
                                                                                     'NA',
                                                                                     'Po',
                                                                                     'Fa',
                                                                                     'TA',
                                                                                     'Gd',
                                                                                     'Ex'],
                                                                                    ['Mix',
                                                                                     'FuseP',
                                                                                     'FuseF',
                                                                                     'FuseA',
                                                                                     'SBrkr'],
                                                                                    ['Po',
                                                                                     'Fa',
                                                                                     'TA',...
                                                                                    ['Sal',
                                                                                     'Sev',
                                                                                     'Maj2',
                                                                                     'Maj1',
                                                                                     'Mod',
                                                                                     'Min2',
                                                                                     'Min1',
                                                                                     'Typ']]))])),
                                       ('decisiontreeclassifier',
                                        DecisionTreeClassifier())]),
             param_grid={'decisiontreeclassifier__criterion': ['gini',
                                                               'entropy'],
                         'decisiontreeclassifier__max_depth': range(2, 8, 2),
        

##### **Check the Onehot Data**

In [50]:
X_cat_onehot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 30 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   MSZoning       1460 non-null   object
 1   Condition1     1460 non-null   object
 2   Heating        1460 non-null   object
 3   Street         1460 non-null   object
 4   CentralAir     1460 non-null   object
 5   Foundation     1460 non-null   object
 6   LotShape       1460 non-null   object
 7   LandContour    1460 non-null   object
 8   Utilities      1460 non-null   object
 9   LotConfig      1460 non-null   object
 10  LandSlope      1460 non-null   object
 11  Neighborhood   1460 non-null   object
 12  Condition2     1460 non-null   object
 13  BldgType       1460 non-null   object
 14  HouseStyle     1460 non-null   object
 15  RoofStyle      1460 non-null   object
 16  RoofMatl       1460 non-null   object
 17  Exterior1st    1460 non-null   object
 18  Exterior2nd    1460 non-null

In [51]:
X_cat_onehot.MSSubClass.value_counts()             

# adjust: Condition1, Exterior1st: 17, Exterior2nd: 18, GarageYrBlt: 28,  

# drop: Street, CentralAir, Heating: 2, Utilities: 8, RoofMatl: 16, YearBuilt: 26, GarageYrBlt: 28, Condition2: 12

20     536
60     299
50     144
120     87
30      69
160     63
70      60
80      58
90      52
190     30
85      20
75      16
45      12
180     10
40       4
Name: MSSubClass, dtype: int64

In [52]:
X_cat_onehot[X_cat_onehot.Exterior1st == 'Stone']

,MSZoning,Condition1,Heating,Street,CentralAir,Foundation,LotShape,LandContour,Utilities,LotConfig,...,GarageType,PavedDrive,SaleType,SaleCondition,MoSold,YrSold,YearBuilt,YearRemodAdd,GarageYrBlt,MSSubClass
1024,RL,Norm,GasA,Pave,Y,CBlock,IR1,Lvl,AllPub,Corner,...,Attchd,Y,COD,Abnorml,5,2008,1976,1976,1976.0,20
1152,RL,Norm,GasA,Pave,Y,PConc,IR1,Lvl,AllPub,Inside,...,Attchd,Y,WD,Abnorml,7,2006,1956,2004,1956.0,20


In [53]:
X_cat_onehot.loc[1187, 'Exterior1st'] = 'AsphShn'
X_cat_onehot.loc[1370, 'Exterior1st'] = 'AsphShn'
X_cat_onehot.loc[1024, 'Exterior1st'] = 'BrkComm'
X_cat_onehot.loc[1152, 'Exterior1st'] = 'BrkComm'
X_cat_onehot.loc[1370, 'Exterior2nd'] = 'AsphShn'
X_cat_onehot.loc[595, 'Exterior2nd'] = 'AsphShn'
X_cat_onehot.loc[1230, 'RoofStyle'] = 'Mansard'
X_cat_onehot.loc[1270, 'RoofStyle'] = 'Mansard'

X_cat_onehot = X_cat_onehot.drop(['Heating', 'Street', 'CentralAir', 'Utilities', \
                                  'RoofMatl', 'YearBuilt', 'GarageYrBlt', 'Condition2'], axis=1)

X_cat_onehot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   MSZoning       1460 non-null   object
 1   Condition1     1460 non-null   object
 2   Foundation     1460 non-null   object
 3   LotShape       1460 non-null   object
 4   LandContour    1460 non-null   object
 5   LotConfig      1460 non-null   object
 6   LandSlope      1460 non-null   object
 7   Neighborhood   1460 non-null   object
 8   BldgType       1460 non-null   object
 9   HouseStyle     1460 non-null   object
 10  RoofStyle      1460 non-null   object
 11  Exterior1st    1460 non-null   object
 12  Exterior2nd    1460 non-null   object
 13  MasVnrType     1452 non-null   object
 14  GarageType     1379 non-null   object
 15  PavedDrive     1460 non-null   object
 16  SaleType       1460 non-null   object
 17  SaleCondition  1460 non-null   object
 18  MoSold         1460 non-null

In [54]:
X_cat_onehot['Date'] = X_cat_onehot['MoSold'].str[:] + X_cat_onehot['YrSold'].str[:]
X_cat_onehot.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 23 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   MSZoning       1460 non-null   object
 1   Condition1     1460 non-null   object
 2   Foundation     1460 non-null   object
 3   LotShape       1460 non-null   object
 4   LandContour    1460 non-null   object
 5   LotConfig      1460 non-null   object
 6   LandSlope      1460 non-null   object
 7   Neighborhood   1460 non-null   object
 8   BldgType       1460 non-null   object
 9   HouseStyle     1460 non-null   object
 10  RoofStyle      1460 non-null   object
 11  Exterior1st    1460 non-null   object
 12  Exterior2nd    1460 non-null   object
 13  MasVnrType     1452 non-null   object
 14  GarageType     1379 non-null   object
 15  PavedDrive     1460 non-null   object
 16  SaleType       1460 non-null   object
 17  SaleCondition  1460 non-null   object
 18  MoSold         1460 non-null

In [55]:
X_cat_onehot = X_cat_onehot.drop(['MoSold', 'YrSold', 'YearRemodAdd'], axis=1)
X_cat_onehot

,MSZoning,Condition1,Foundation,LotShape,LandContour,LotConfig,LandSlope,Neighborhood,BldgType,HouseStyle,RoofStyle,Exterior1st,Exterior2nd,MasVnrType,GarageType,PavedDrive,SaleType,SaleCondition,MSSubClass,Date
0,RL,Norm,PConc,Reg,Lvl,Inside,Gtl,CollgCr,1Fam,2Story,Gable,VinylSd,VinylSd,BrkFace,Attchd,Y,WD,Normal,60,22008
1,RL,Feedr,CBlock,Reg,Lvl,FR2,Gtl,Veenker,1Fam,1Story,Gable,MetalSd,MetalSd,None,Attchd,Y,WD,Normal,20,52007
2,RL,Norm,PConc,IR1,Lvl,Inside,Gtl,CollgCr,1Fam,2Story,Gable,VinylSd,VinylSd,BrkFace,Attchd,Y,WD,Normal,60,92008
3,RL,Norm,BrkTil,IR1,Lvl,Corner,Gtl,Crawfor,1Fam,2Story,Gable,Wd Sdng,Wd Shng,None,Detchd,Y,WD,Abnorml,70,22006
4,RL,Norm,PConc,IR1,Lvl,FR2,Gtl,NoRidge,1Fam,2Story,Gable,VinylSd,VinylSd,BrkFace,Attchd,Y,WD,Normal,60,122008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,RL,Norm,PConc,Reg,Lvl,Inside,Gtl,Gilbert,1Fam,2Story,Gable,VinylSd,VinylSd,None,Attchd,Y,WD,Normal,60,82007
1456,RL,Norm,CBlock,Reg,Lvl,Inside,Gtl,NWAmes,1Fam,1Story,Gable,Plywood,Plywood,Stone,Attchd,Y,WD,Normal,20,22010
1457,RL,Norm,Stone,Reg,Lvl,Inside,Gtl,Crawfor,1Fam,2Story,Gable,CemntBd,CmentBd,None,Attchd,Y,WD,Normal,70,52010
1458,RL,Norm,CBlock,Reg,Lvl,Inside,Gtl,NAmes,1Fam,1Story,Hip,MetalSd,MetalSd,None,Attchd,Y,WD,Normal,20,42010


In [56]:
cat_onehot_pipeline = make_pipeline(SimpleImputer(strategy="constant", fill_value="N_A"),\
                                    OneHotEncoder(drop='first', handle_unknown="ignore"))
cat_onehot_pipeline

Pipeline(steps=[('simpleimputer',
                 SimpleImputer(fill_value='N_A', strategy='constant')),
                ('onehotencoder',
                 OneHotEncoder(drop='first', handle_unknown='ignore'))])

In [57]:
full_pipeline = make_pipeline(cat_onehot_pipeline, DecisionTreeClassifier())
full_pipeline

Pipeline(steps=[('pipeline',
                 Pipeline(steps=[('simpleimputer',
                                  SimpleImputer(fill_value='N_A',
                                                strategy='constant')),
                                 ('onehotencoder',
                                  OneHotEncoder(drop='first',
                                                handle_unknown='ignore'))])),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [58]:
param_grid = {'decisiontreeclassifier__criterion': ['gini', 'entropy'],
              'decisiontreeclassifier__max_depth': range(2, 8, 2),
              'decisiontreeclassifier__min_samples_leaf': range(3, 5, 2),
              'decisiontreeclassifier__min_samples_split': range(5, 15, 5)
             }
param_grid

{'decisiontreeclassifier__criterion': ['gini', 'entropy'],
 'decisiontreeclassifier__max_depth': range(2, 8, 2),
 'decisiontreeclassifier__min_samples_leaf': range(3, 5, 2),
 'decisiontreeclassifier__min_samples_split': range(5, 15, 5)}

In [59]:
search = GridSearchCV(full_pipeline, param_grid, cv=5, verbose=1)
search

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('pipeline',
                                        Pipeline(steps=[('simpleimputer',
                                                         SimpleImputer(fill_value='N_A',
                                                                       strategy='constant')),
                                                        ('onehotencoder',
                                                         OneHotEncoder(drop='first',
                                                                       handle_unknown='ignore'))])),
                                       ('decisiontreeclassifier',
                                        DecisionTreeClassifier())]),
             param_grid={'decisiontreeclassifier__criterion': ['gini',
                                                               'entropy'],
                         'decisiontreeclassifier__max_depth': range(2, 8, 2),
                         'decisiontreeclassifier__min_samples_leaf': range(3, 5, 2),
                         'decisiontreeclassifier__min_samples_split': range(5, 15, 5)},
             verbose=1)

In [60]:
search.fit(X_cat_onehot, y)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('pipeline',
                                        Pipeline(steps=[('simpleimputer',
                                                         SimpleImputer(fill_value='N_A',
                                                                       strategy='constant')),
                                                        ('onehotencoder',
                                                         OneHotEncoder(drop='first',
                                                                       handle_unknown='ignore'))])),
                                       ('decisiontreeclassifier',
                                        DecisionTreeClassifier())]),
             param_grid={'decisiontreeclassifier__criterion': ['gini',
                                                               'entropy'],
                         'decisiontreeclassifier__max_depth': range(2, 8, 2),
                         'decisiontreeclassifier__min_samples_leaf': range(3, 5, 2),
                         'decisiontreeclassifier__min_samples_split': range(5, 15, 5)},
             verbose=1)

In [61]:
X_train = pd.concat([X_num, X_cat_ord, X_cat_onehot], axis=1)
X_train

,LotArea,LotFrontage,TotalBsmtSF,BedroomAbvGr,Fireplaces,GarageCars,WoodDeckSF,OverallQual,OverallCond,MasVnrArea,...,RoofStyle,Exterior1st,Exterior2nd,MasVnrType,GarageType,PavedDrive,SaleType,SaleCondition,MSSubClass,Date
0,8450,65.0,856,3,0,2,0,7,5,196.0,...,Gable,VinylSd,VinylSd,BrkFace,Attchd,Y,WD,Normal,60,22008
1,9600,80.0,1262,3,1,2,298,6,8,0.0,...,Gable,MetalSd,MetalSd,None,Attchd,Y,WD,Normal,20,52007
2,11250,68.0,920,3,1,2,0,7,5,162.0,...,Gable,VinylSd,VinylSd,BrkFace,Attchd,Y,WD,Normal,60,92008
3,9550,60.0,756,3,1,3,0,7,5,0.0,...,Gable,Wd Sdng,Wd Shng,None,Detchd,Y,WD,Abnorml,70,22006
4,14260,84.0,1145,4,1,3,192,8,5,350.0,...,Gable,VinylSd,VinylSd,BrkFace,Attchd,Y,WD,Normal,60,122008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,7917,62.0,953,3,1,2,0,6,5,0.0,...,Gable,VinylSd,VinylSd,None,Attchd,Y,WD,Normal,60,82007
1456,13175,85.0,1542,3,2,2,349,6,6,119.0,...,Gable,Plywood,Plywood,Stone,Attchd,Y,WD,Normal,20,22010
1457,9042,66.0,1152,4,2,1,0,7,9,0.0,...,Gable,CemntBd,CmentBd,None,Attchd,Y,WD,Normal,70,52010
1458,9717,68.0,1078,2,0,1,366,5,6,0.0,...,Hip,MetalSd,MetalSd,None,Attchd,Y,WD,Normal,20,42010


##### **Initiate Column Transformer and Build Pipeline with DecisionClassifier**

In [62]:
processor = ColumnTransformer(transformers=[('cat_onehot', cat_onehot_pipeline, X_cat_onehot.columns),
                                            ('num_pipe', numeric_pipeline, X_num.columns),
                                            ('cat_ord', cat_ordinal_pipeline, X_cat_ord.columns),
                                           ]
                             )
processor

ColumnTransformer(transformers=[('cat_onehot',
                                 Pipeline(steps=[('simpleimputer',
                                                  SimpleImputer(fill_value='N_A',
                                                                strategy='constant')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'))]),
                                 Index(['MSZoning', 'Condition1', 'Foundation', 'LotShape', 'LandContour',
       'LotConfig', 'LandSlope', 'Neighborhood', 'BldgType', 'HouseStyle',
       'RoofStyle', 'E...
                                                                              'NA',
                                                                              'Po',
                                                                              'Fa',
                                                                              'TA',
                                                                              'Gd',
                                                                              'Ex'],
                                                                             ['Po',
                                                                              'Fa',
                                                                              'TA',
                                                                              'Gd',
                                                                              'Ex'],
                                                                             ['Po',
                                                                              'Fa',
                                                                              'TA',
                                                                              'Gd',
                                                                              'Ex'],
                                                                             ['Sal',
                                                                              'Sev',
                                                                              'Maj2',
                                                                              'Maj1',
                                                                              'Mod',
                                                                              'Min2',
                                                                              'Min1',
                                                                              'Typ']]))]),
                                 Index(['GarageCond', 'GarageFinish', 'GarageQual', 'Electrical', 'HeatingQC',
       'BsmtFinType2', 'BsmtFinType1', 'BsmtExposure', 'KitchenQual',
       'BsmtQual', 'BsmtCond', 'ExterCond', 'ExterQual', 'Functional'],
      dtype='object'))])

In [63]:
full_pipeline = make_pipeline(processor, DecisionTreeClassifier())
full_pipeline

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('cat_onehot',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(fill_value='N_A',
                                                                                 strategy='constant')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder(drop='first',
                                                                                 handle_unknown='ignore'))]),
                                                  Index(['MSZoning', 'Condition1', 'Foundation', 'LotShape', 'LandContour',
       'LotConfig', 'LandSlope', 'Neighborhood', '...
                                                                                              ['Po',
                                                                                               'Fa',
                                                                                               'TA',
                                                                                               'Gd',
                                                                                               'Ex'],
                                                                                              ['Sal',
                                                                                               'Sev',
                                                                                               'Maj2',
                                                                                               'Maj1',
                                                                                               'Mod',
                                                                                               'Min2',
                                                                                               'Min1',
                                                                                               'Typ']]))]),
                                                  Index(['GarageCond', 'GarageFinish', 'GarageQual', 'Electrical', 'HeatingQC',
       'BsmtFinType2', 'BsmtFinType1', 'BsmtExposure', 'KitchenQual',
       'BsmtQual', 'BsmtCond', 'ExterCond', 'ExterQual', 'Functional'],
      dtype='object'))])),
                ('decisiontreeclassifier', DecisionTreeClassifier())])

In [64]:
param_grid = {'columntransformer__num_pipe__simpleimputer__strategy': ['mean', 'median'],
              'decisiontreeclassifier__criterion': ['gini', 'entropy'],
              'decisiontreeclassifier__max_depth': range(2, 15),
              'decisiontreeclassifier__min_samples_leaf': range(2, 10),
              'decisiontreeclassifier__min_samples_split': range(2, 25)
             }
param_grid

{'columntransformer__num_pipe__simpleimputer__strategy': ['mean', 'median'],
 'decisiontreeclassifier__criterion': ['gini', 'entropy'],
 'decisiontreeclassifier__max_depth': range(2, 15),
 'decisiontreeclassifier__min_samples_leaf': range(2, 10),
 'decisiontreeclassifier__min_samples_split': range(2, 25)}

In [65]:
search = GridSearchCV(full_pipeline, param_grid, cv=5, verbose=1)
search

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('cat_onehot',
                                                                         Pipeline(steps=[('simpleimputer',
                                                                                          SimpleImputer(fill_value='N_A',
                                                                                                        strategy='constant')),
                                                                                         ('onehotencoder',
                                                                                          OneHotEncoder(drop='first',
                                                                                                        handle_unknown='ignore'))]),
                                                                         Index(['MSZoning', 'Condition1', 'Foundation', 'LotShape', 'LandContour',
       'LotConfig',...
                                       ('decisiontreeclassifier',
                                        DecisionTreeClassifier())]),
             param_grid={'columntransformer__num_pipe__simpleimputer__strategy': ['mean',
                                                                                  'median'],
                         'decisiontreeclassifier__criterion': ['gini',
                                                               'entropy'],
                         'decisiontreeclassifier__max_depth': range(2, 15),
                         'decisiontreeclassifier__min_samples_leaf': range(2, 10),
                         'decisiontreeclassifier__min_samples_split': range(2, 25)},
             verbose=1)

In [66]:
start_time = time.time()

search.fit(X_train, y)

end_time = time.time()

print(end_time - start_time)

Fitting 5 folds for each of 9568 candidates, totalling 47840 fits
3779.6687426567078


In [67]:
search.best_params_

{'columntransformer__num_pipe__simpleimputer__strategy': 'mean',
 'decisiontreeclassifier__criterion': 'gini',
 'decisiontreeclassifier__max_depth': 13,
 'decisiontreeclassifier__min_samples_leaf': 3,
 'decisiontreeclassifier__min_samples_split': 14}

In [68]:
search.best_score_

0.9301369863013699

In [69]:
y_pred = search.predict(X_train)
y_pred

array([0, 0, 0, ..., 1, 0, 0])

In [70]:
accuracy_score(y_pred, y)

0.9698630136986301